In [113]:
from login import config

params = config()

In [70]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

# Creating a webdriver instance
driver = webdriver.Chrome()
# This instance will be used to log into LinkedIn

# Opening linkedIn's login page
driver.get("https://linkedin.com/uas/login")

# waiting for the page to load
time.sleep(5)

# entering username
username = driver.find_element(By.ID, "username")
# In case of an error, try changing the element tag used here.

# Enter Your Email Address
username.send_keys(params['user'])

# entering password
pword = driver.find_element(By.ID, "password")
# In case of an error, try changing the element tag used here.

# Enter Your Password
pword.send_keys(params['password'])

# Clicking on the log in button
# Format (syntax) of writing XPath --> //tagname[@attribute='value']
driver.find_element(By.XPATH, "//button[@type='submit']").click()
# In case of an error, try changing the XPath used here.

In [77]:
def search_job_in_page(job_list, cur_page_num):
    job_name = job_list.find_all('li', {'class':'ember-view'})
    for row in job_name:
        try:
            job_title = row.find('a').contents[0].strip()
            company = row.find('div', {'class':'job-card-container__company-name'}).contents[0].strip()
            level = job_list.find('li', {'class':'jobs-unified-top-card__job-insight'}).find('span').contents[2]
            print('Job Title: ',job_title)
            print('Company: ', company)
            print('Level: ',level)
            print()
        except: 
            try:
                p_num = int(row.find('button').attrs.get('aria-label')[-1:])
                if cur_page_num<p_num:
                    cur_page_num=p_num
                    break
                elif cur_page_num>=p_num:
                    continue
            except:
                continue
    return cur_page_num

In [78]:
def get_job_titles_company(last_page):
    for i in range(1, last_page+1):
        src = driver.page_source

        # Now using beautiful soup
        soup = BeautifulSoup(src, 'html5lib')

        job_list = soup.find('div', {'class':'scaffold-layout__list-detail-inner'})
        print('||Listing all job lists in page ', i+'||')
        next_page_num = search_job_in_page(job_list,i)
        print()
        driver.find_element(By.XPATH, "//button[@aria-label='Page "+str(next_page_num)+"']").click()
        time.sleep(7)
    return next_page_num

In [79]:
recomend_url= 'https://www.linkedin.com/jobs/collections/recommended?lipi=urn%3Ali%3Apage%3Ad_flagship3_job_home%3BobHSV5i9RA6yU%2F%2F5gLL1pQ%3D%3D'
jobs_url = "https://www.linkedin.com/jobs/"
    
# driver.get(jobs_url)

driver.get(recomend_url)
get_job_titles_company(2)

Listing all job lists in page  1
Job Title:  Junior Data Analyst
Company:  Centurion
Level:  Full-time · Mid-Senior level

Job Title:  Data Analyst – Data Team (Bangkok based, relocation provided)
Company:  Agoda
Level:  Full-time · Mid-Senior level

Job Title:  Supply Chain Analyst / Data Analyst
Company:  Michael Page
Level:  Full-time · Mid-Senior level

Job Title:  Analyst - RNA
Company:  WNS
Level:  Full-time · Mid-Senior level

Job Title:  Insight Analyst, Unloan, x15 Ventures
Company:  Commonwealth Bank
Level:  Full-time · Mid-Senior level

Job Title:  Business Intelligence Analyst
Company:  SustainAbility Consulting
Level:  Full-time · Mid-Senior level

Job Title:  Business Intelligence Tester
Company:  Peoplebank
Level:  Full-time · Mid-Senior level


Listing all job lists in page  2
Job Title:  ANZ Business Insights, Machine Learning / Data Engineer
Company:  The Aicila Group - International Recruitment & Consulting Specialists
Level:  Full-time

Job Title:  Professional, Dat

3

In [115]:
src = driver.page_source

# Now using beautiful soup
soup = BeautifulSoup(src, 'html5lib')

job_list = soup.find('div', {'class':'scaffold-layout__list-detail-inner'})

text = job_list.find('div', {'class':'jobs-description__content jobs-description-content'}).find('span').getText().strip()

In [112]:
import openai

In [114]:
openai.api_key = params['api_key']

In [116]:
system_msg="You are a career coach who understands job requirements"
user_msg="List 5 key skills to be a great fit in this job description "+text
response = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                        messages=[{"role": "system", "content": system_msg},
                                         {"role": "user", "content": user_msg}])

In [118]:
print(response["choices"][0]["message"]["content"])

1. Strong analytical skills: The role requires the ability to collect, compile, analyze, and interpret data. A great fit would have a strong analytical mindset and be able to extract insights from data to support decision-making.
2. Proficiency in data manipulation and visualization tools: The candidate should have excellent skills in Microsoft Excel and Microsoft Power BI. This includes the ability to manipulate data, perform statistical analysis, and create visual representations such as dashboards and reports.
3. Attention to detail and accuracy: Given the importance of data quality, it is crucial for the candidate to have a high level of attention to detail and the ability to maintain accuracy in data collection, analysis, and reporting.
4. Problem-solving and continuous improvement mindset: The role involves identifying potential improvements in reporting processes and automating tasks. Therefore, a great fit would be someone who has a proactive problem-solving mindset and is cons